In [1]:
from problem import get_train_data
from problem import get_test_data

X_df, y_array = get_train_data()
X_test_df, y_test_array = get_test_data()

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit

In [3]:
award = pd.read_csv('data/award_notices_RAMP.csv.zip', compression='zip')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# # obtain features from award
# award['Name_processed'] = award['incumbent_name'].str.lower()
# award['Name_processed'] = award['Name_processed'].str.replace('[^\w]','')
# award['End_of_call_date'] = pd.to_datetime(award['End_of_call_date'], format='%Y-%m-%d', errors='coerce')
# award['End_of_call_year'] = award['End_of_call_date'].dt.year
# award_features = award.groupby(['Name_processed','End_of_call_year'])['amount'].agg(['count','sum'])

# def zipcodes(X):
#     zipcode_nums = pd.to_numeric(X['Zipcode'], errors='coerce')
#     return zipcode_nums.values[:, np.newaxis]
# zipcode_transformer = FunctionTransformer(zipcodes, validate=False)

# numeric_transformer = Pipeline(steps=[
#     ('impute', SimpleImputer(strategy='median'))])

# def process_date(X):
#     date = pd.to_datetime(X['Fiscal_year_end_date'], format='%Y-%m-%d')
#     return np.c_[date.dt.year, date.dt.month, date.dt.day]
# date_transformer = FunctionTransformer(process_date, validate=False)

# def process_APE(X):
#     APE = X['Activity_code (APE)'].str[:2]
#     return pd.to_numeric(APE).values[:, np.newaxis]
# APE_transformer = FunctionTransformer(process_APE, validate=False)

# def merge_naive(X):
#     X['Name'] = X['Name'].str.lower()     
#     X['Name'] = X['Name'].str.replace('[^\w]','')
#     df = pd.merge(X, award_features, left_on=['Name','Year'], 
#                   right_on=['Name_processed','End_of_call_year'], how='left')
#     return df[['count','sum']]
# merge_transformer = FunctionTransformer(merge_naive, validate=False)

# num_cols = ['Legal_ID', 'Headcount', 
#             'Fiscal_year_duration_in_months', 'Year']
# zipcode_col = ['Zipcode']
# date_cols = ['Fiscal_year_end_date']
# APE_col = ['Activity_code (APE)']
# merge_col = ['Name','Year']
# drop_cols = ['Address', 'City']

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('zipcode', make_pipeline(zipcode_transformer, SimpleImputer(strategy='median')), zipcode_col),
#         ('num', numeric_transformer, num_cols),
#         ('date', make_pipeline(date_transformer, SimpleImputer(strategy='median')), date_cols),
#         ('APE', make_pipeline(APE_transformer, SimpleImputer(strategy='median')), APE_col),
#         ('merge', make_pipeline(merge_transformer, SimpleImputer(strategy='median')), merge_col),
#         ('drop cols', 'drop', drop_cols),
#         ])

In [5]:
from sklearn.metrics import make_scorer

def loss(y_true, y_pred):
    
    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    true = np.maximum(5., np.log10(np.maximum(1., y_true)))
    pred = np.maximum(5., np.log10(np.maximum(1., y_pred)))
    
    loss = np.mean(np.abs(true - pred))
    
    return loss
    
fan_loss = make_scorer(loss, greater_is_better=False)

In [22]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=10)

In [6]:
import category_encoders as ce

In [152]:
X_toy = X_df[-1000:]

In [161]:
X_df['Activity_code (APE)'].str[-1].value_counts()

Z    804086
A    339634
B    216133
C     96920
D     16787
F      2395
E       887
J       279
G       133
L        47
S        30
V        29
K        29
N        25
W        21
H        17
1        16
T        14
3        13
P        13
M         9
U         8
6         6
R         6
5         6
X         4
0         4
9         4
2         3
Name: Activity_code (APE), dtype: int64

In [163]:
def process_APE_char(X):
    char_map = {'Z':9,
                'A':8,
                'B':7,
                'C':6,
                'D':5,
                'F':4,
                'E':3,
                'J':2,
                'G':1}
    APEc = X['Activity_code (APE)'].str[-1]
    return np.c_[[char_map[c] if c in char_map else 0 for c in APEc]]

In [164]:
process_APE_char(X_toy)

array([[8],
       [8],
       [8],
       [9],
       [9],
       [9],
       [8],
       [8],
       [8],
       [8],
       [8],
       [8],
       [8],
       [7],
       [7],
       [7],
       [7],
       [7],
       [9],
       [9],
       [9],
       [8],
       [8],
       [8],
       [8],
       [7],
       [7],
       [7],
       [8],
       [8],
       [8],
       [8],
       [8],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [8],
       [8],
       [8],
       [8],
       [9],
       [9],
       [9],
       [8],
       [8],
       [8],
       [8],
       [9],
       [9],
       [9],
       [9],
       [7],
       [7],
       [7],
       [7],
       [7],
       [8],
       [8],
       [8],
       [8],
       [8],
       [8],
       [8],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
       [9],
    

In [149]:
X_toy['Activity_code (APE)'][-1:]

999    9329Z
Name: Activity_code (APE), dtype: object

In [10]:
encoder = ce.OneHotEncoder(cols=['Year'])
encoder.fit_transform(X_toy['Year']).values

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0]], dtype=int64)

In [13]:
from sklearn.preprocessing import scale,StandardScaler

In [6]:
award['Name_processed'] = award['incumbent_name'].str.lower()
award['Name_processed'] = award['Name_processed'].str.replace('[^\w]','')
award['End_of_call_date'] = pd.to_datetime(award['End_of_call_date'], format='%Y-%m-%d', errors='coerce')
award['End_of_call_year'] = award['End_of_call_date'].dt.year
award_features = award.groupby(['Name_processed','End_of_call_year'])['amount'].agg(['count','sum'])

In [7]:
def process_APE(X):
    APE1 = X['Activity_code (APE)'].str[:2]
    APE2 = X['Activity_code (APE)'].str[:4]
    char_map = {'Z':9.,
                'A':8.,
                'B':7.,
                'C':6.,
                'D':5.,
                'F':4.,
                'E':3.,
                'J':2.,
                'G':1.}
    APEc = X['Activity_code (APE)'].str[-1]
    APEc = [char_map[c] if c in char_map else 0. for c in APEc]
    return np.c_[pd.to_numeric(APE1, errors='coerce').values,
                 pd.to_numeric(APE2, errors='coerce').values,
                 APEc
                 ]
APE_transformer = FunctionTransformer(process_APE, validate=False)

def merge_naive(X):
    X['Name'] = X['Name'].str.lower()     
    X['Name'] = X['Name'].str.replace('[^\w]','')
    df = pd.merge(X, award_features, left_on=['Name','Year'], 
                  right_on=['Name_processed','End_of_call_year'], how='left')
    return df[['count','sum']]
merge_award_transformer = FunctionTransformer(merge_naive, validate=False)

def merge_headcounts(X):  
    X['Name'] = X['Name'].str.lower()     
    X['Name'] = X['Name'].str.replace('[^\w]','')
    
    head_att = ['mean','max','min' ]
    headcount_features = X.groupby(['Name'])['Headcount'].agg(head_att)    
    df = pd.merge(X, headcount_features, left_on=['Name'], 
                right_on=['Name'], how='left')
    return df[head_att]
merge_head_transformer = FunctionTransformer(merge_headcounts, validate=False)

def encode_year(X):
    return np.c_[(X['Year']==2013.) * 1.,
                 (X['Year']==2014.) * 1.,
                 (X['Year']==2015.) * 1.,
                 (X['Year']==2016.) * 1.,
                 (X['Year']==2017.) * 1.,
                 (X['Year']==2018.) * 1.]
encode_year_transformer = FunctionTransformer(encode_year, validate=False)

num_cols = ['Legal_ID', 'Headcount', 'Year']
APE_col = ['Activity_code (APE)']
merge_award_cols = ['Name','Year']
merge_headcount_col = ['Name','Headcount']
id_col = ['Legal_ID']
head_col = ['Headcount']
year_col = ['Year']

numeric_transformer = Pipeline(steps=[
            ('impute', SimpleImputer(strategy='median')),
            #('scale', StandardScaler())
            ])

preprocessor = ColumnTransformer(
    transformers=[
        #('num', numeric_transformer, num_cols),
        ('ID', make_pipeline(SimpleImputer(strategy='mean')), id_col),
        ('Headcount', make_pipeline(SimpleImputer(strategy='median')), head_col),
        ('Year', make_pipeline(SimpleImputer(strategy='median')), year_col),
        #('Year encoders', make_pipeline(encode_year_transformer, SimpleImputer(strategy='median')), year_col),
        ('APE', make_pipeline(APE_transformer, SimpleImputer(strategy='median')), APE_col),
        ('merge headcounts', make_pipeline(merge_head_transformer, SimpleImputer(strategy='median')), merge_headcount_col),
        ('merge awards', make_pipeline(merge_award_transformer, SimpleImputer(strategy='median')), merge_award_cols),
        ])

## Transform data

In [8]:
X_num_train = preprocessor.fit_transform(X_df)
X_num_test = preprocessor.fit_transform(X_test_df)
print("Train data shape:")
print(X_num_train.shape)
print("Test data shape:")
print(X_num_test.shape)

Train data shape:
(1495948, 11)
Test data shape:
(520966, 11)


In [35]:
n_train = -1
n_test = -1
X_train = X_num_train[:n_train]
X_test = X_num_test[:n_test]
y_train = y_array[:n_train]
y_test = y_test_array[:n_test]

## Baseline - 10000 train - 10000 test -  mean=0.455717844843999 -std=0.004

In [11]:
from sklearn.ensemble import RandomForestRegressor

In [37]:
regressor = RandomForestRegressor(n_estimators=10)

n_try = 1
loss_ = []
for _ in range(n_try):
    regressor.fit(X_train, np.log10(np.maximum(1,y_train)))

    y_pred = 10 ** regressor.predict(X_test)
    loss_.append(loss(y_test, y_pred))
    
print("mean:",np.mean(loss_))
print("std:",np.std(loss_))

mean: 0.33539629333072046
std: 0.0


In [27]:
regressor = RandomForestRegressor(n_estimators=10)

n_try = 10
loss_ = []
for _ in range(n_try):
    regressor.fit(X_train, np.log1p(np.maximum(1,y_train)))

    y_pred = np.exp(regressor.predict(X_test))
    loss_.append(loss(y_test, y_pred))
    
print("mean:",np.mean(loss_))
print("std:",np.std(loss_))

mean: 0.45728519563597547
std: 0.015222342140296857


In [129]:
y_toy =y_train[:10] 

In [135]:
np.maximum(y_toy,10000000000)

array([1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10])

In [38]:
xg_boost = xgb.XGBRegressor( learning_rate=0.01,
                         n_estimators=2000,
                         #max_depth=4, min_child_weight=1,
                         #gamma=0.6, subsample=0.7,
                         #colsample_bytree=0.2, nthread=-1,
                         #scale_pos_weight=1, seed=27,
                         #reg_alpha=0.00006
                       )
xg_boost.fit(X_train, np.log10(np.maximum(y_train,1)))

y_pred = 10 ** (xg_boost.predict(X_test))
print(loss(y_test, y_pred))

0.31715693138700285


In [18]:
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor

In [29]:
xg_boost = xgb.XGBRegressor()
regressor = AdaBoostRegressor(xg_boost)
regressor.fit(X_train, np.log10(np.maximum(y_train,1)))

y_pred = 10 ** (regressor.predict(X_test))
print(loss(y_test, y_pred))

0.42888883590792337


## Grid search

In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

random_classifier = RandomForestRegressor()
#cv = GroupShuffleSplit(n_splits=8, test_size=0.25)

parameters = { 'max_features': np.arange(7,10),
               'n_estimators': [8,9,10],
               'min_samples_leaf': [10,20,30,
                                    #100,200,500
                                   ]}

random_grid = GridSearchCV(random_classifier, parameters, scoring=fan_loss)

In [42]:
print("# Tuning hyper-parameters:")
print()

random_grid.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(random_grid.best_params_)
print()
print("Grid scores on development set:")
print()
means = random_grid.cv_results_['mean_test_score']
stds = random_grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, random_grid.predict(X_test)
print(loss(y_true, y_pred))


# Tuning hyper-parameters:



D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters set found on development set:

{'max_features': 8, 'min_samples_leaf': 10, 'n_estimators': 10}

Grid scores on development set:

-0.498 (+/-0.097) for {'max_features': 7, 'min_samples_leaf': 10, 'n_estimators': 8}
-0.487 (+/-0.107) for {'max_features': 7, 'min_samples_leaf': 10, 'n_estimators': 9}
-0.486 (+/-0.076) for {'max_features': 7, 'min_samples_leaf': 10, 'n_estimators': 10}
-0.520 (+/-0.109) for {'max_features': 7, 'min_samples_leaf': 20, 'n_estimators': 8}
-0.507 (+/-0.118) for {'max_features': 7, 'min_samples_leaf': 20, 'n_estimators': 9}
-0.495 (+/-0.114) for {'max_features': 7, 'min_samples_leaf': 20, 'n_estimators': 10}
-0.502 (+/-0.112) for {'max_features': 7, 'min_samples_leaf': 30, 'n_estimators': 8}
-0.510 (+/-0.129) for {'max_features': 7, 'min_samples_leaf': 30, 'n_estimators': 9}
-0.507 (+/-0.129) for {'max_features': 7, 'min_samples_leaf': 30, 'n_estimators': 10}
-0.491 (+/-0.115) for {'max_features': 8, 'min_samples_leaf': 10, 'n_estimators': 8}
-0

## Test

In [31]:
y_train[0]

104334380.0

In [92]:
regressor = RandomForestRegressor(n_estimators=10,max_features = 8, min_samples_leaf = 10, max_depth = 10)

clf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', regressor)])

cv = GroupShuffleSplit(n_splits=8, test_size=0.25)

n_samples = 1000000
#n_samples = len(X_df)
scores_Xdf = -cross_val_score(clf, X_df[:n_samples], y_array[:n_samples], cv=cv, scoring=fan_loss, groups=X_df['Legal_ID'][:n_samples], n_jobs=2)

print("mean: %e (+/- %e)" % (scores_Xdf.mean(), scores_Xdf.std()))

mean: 5.790908e-01 (+/- 1.131855e-02)


In [ ]:
for max_features_ in np.arange(5,11):
    for n_estimators_ in np.arange(7,11):
        for min_samples_leaf_ in [1,10,20]:
            for max_depth_ in [None, 10,25,50]:
                regressor = RandomForestRegressor(n_estimators=n_estimators_,
                                                  max_features = max_features_, 
                                                  min_samples_leaf = min_samples_leaf_, 
                                                  max_depth = max_depth_)

                clf = Pipeline(steps=[
                    ('preprocessing', preprocessor),
                    ('classifier', regressor)])

                cv = GroupShuffleSplit(n_splits=8, test_size=0.25)

                #n_samples = 1000
                n_samples = len(X_df)
                scores_Xdf = -cross_val_score(clf, X_df[:n_samples], y_array[:n_samples], cv=cv, scoring=fan_loss, groups=X_df['Legal_ID'][:n_samples], n_jobs=2)
                print("===================")
                print("max_features: {} - n_estimators: {} - min_samples_leaf: {} - max_depth: {}".format(max_features_, n_estimators_, min_samples_leaf_,max_depth_))
                print("mean: %e (+/- %e)" % (scores_Xdf.mean(), scores_Xdf.std()))

max_features: 5 - n_estimators: 7 - min_samples_leaf: 1 - max_depth: None
mean: 3.710074e-01 (+/- 9.594759e-04)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 1 - max_depth: 10
mean: 6.297797e-01 (+/- 1.967843e-02)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 1 - max_depth: 25
mean: 3.959337e-01 (+/- 3.453724e-03)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 1 - max_depth: 50
mean: 3.696194e-01 (+/- 7.739701e-04)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 10 - max_depth: None
mean: 3.880310e-01 (+/- 1.249568e-03)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 10 - max_depth: 10
mean: 6.463862e-01 (+/- 2.890824e-02)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 10 - max_depth: 25
mean: 4.078612e-01 (+/- 4.038989e-03)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 10 - max_depth: 50
mean: 3.876052e-01 (+/- 1.700589e-03)
max_features: 5 - n_estimators: 7 - min_samples_leaf: 20 - max_depth: None
mean: 3.994445e-01 (+/- 3.193338e-03)

max_features: 6 - n_estimators: 8 - min_samples_leaf: 1 - max_depth: 50
mean: 3.699066e-01 (+/- 8.894680e-04)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 10 - max_depth: None
mean: 3.848318e-01 (+/- 1.165358e-03)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 10 - max_depth: 10
mean: 6.258759e-01 (+/- 1.523693e-02)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 10 - max_depth: 25
mean: 4.030645e-01 (+/- 3.195959e-03)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 10 - max_depth: 50
mean: 3.855913e-01 (+/- 3.173429e-03)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 20 - max_depth: None
mean: 3.965816e-01 (+/- 4.128837e-03)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 20 - max_depth: 10
mean: 6.045039e-01 (+/- 2.090099e-02)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 20 - max_depth: 25
mean: 4.092159e-01 (+/- 2.091090e-03)
max_features: 6 - n_estimators: 8 - min_samples_leaf: 20 - max_depth: 50
mean: 3.972064e-01 (+/- 2.983869e-03

max_features: 7 - n_estimators: 9 - min_samples_leaf: 10 - max_depth: 25
mean: 4.012507e-01 (+/- 1.903068e-03)
max_features: 7 - n_estimators: 9 - min_samples_leaf: 10 - max_depth: 50
mean: 3.825346e-01 (+/- 7.915259e-04)
max_features: 7 - n_estimators: 9 - min_samples_leaf: 20 - max_depth: None
mean: 3.922634e-01 (+/- 4.496847e-04)
max_features: 7 - n_estimators: 9 - min_samples_leaf: 20 - max_depth: 10
mean: 6.110660e-01 (+/- 3.153389e-02)
max_features: 7 - n_estimators: 9 - min_samples_leaf: 20 - max_depth: 25
mean: 4.068142e-01 (+/- 2.454316e-03)
max_features: 7 - n_estimators: 9 - min_samples_leaf: 20 - max_depth: 50
mean: 3.933162e-01 (+/- 3.954324e-03)
max_features: 7 - n_estimators: 10 - min_samples_leaf: 1 - max_depth: None
mean: 3.710646e-01 (+/- 1.070048e-03)
max_features: 7 - n_estimators: 10 - min_samples_leaf: 1 - max_depth: 10
mean: 6.410534e-01 (+/- 3.106206e-02)
max_features: 7 - n_estimators: 10 - min_samples_leaf: 1 - max_depth: 25
mean: 3.955351e-01 (+/- 4.213551e-0

In [94]:
6*4*3*4

288

In [1]:
!ramp_test_submission --submission haiyen_submission_2

Testing Prediction of annual revenue using FAN
Reading train and test files from ./data ...
Reading cv ...
Training submissions\haiyen_submission_2 ...
CV fold 0
	score  fan error       time
	train       0.12  69.221695
	valid       0.41  17.332377
	test        0.42   9.392591
CV fold 1
	score  fan error       time
	train       0.12  67.652653
	valid       0.42  18.304875
	test        0.42   9.376451
CV fold 2
	score  fan error       time
	train       0.12  67.769182
	valid       0.42  16.910084
	test        0.42   9.808375
CV fold 3
	score  fan error       time
	train       0.13  68.913406
	valid       0.42  17.027937
	test        0.42   9.798610
CV fold 4
	score  fan error       time
	train       0.12  69.383022
	valid       0.42  18.076140
	test        0.43  10.011144
CV fold 5
	score  fan error       time
	train       0.13  69.706588
	valid       0.42  17.974736
	test        0.42   9.865078
CV fold 6
	score  fan error       time
	train       0.12  69.745995
	valid       0.42  17.55

In [2]:
!ramp_test_submission --submission haiyen_submission_3

Testing Prediction of annual revenue using FAN
Reading train and test files from ./data ...
Reading cv ...
Training submissions\haiyen_submission_3 ...
CV fold 0
	score  fan error       time
	train       0.13  70.247458
	valid       0.38  14.722892
	test        0.37   8.500434
CV fold 1
	score  fan error       time
	train       0.13  69.280671
	valid       0.39  14.574807
	test        0.37   8.316836
CV fold 2
	score  fan error       time
	train       0.14  70.303200
	valid       0.39  14.221292
	test        0.37   8.509389
CV fold 3
	score  fan error       time
	train       0.13  68.348161
	valid       0.39  14.417812
	test        0.37   8.504932
CV fold 4
	score  fan error       time
	train       0.14  70.654614
	valid       0.39  14.497395
	test        0.37   8.543313
CV fold 5
	score  fan error       time
	train       0.13  67.768689
	valid       0.38  14.048547
	test        0.37   8.462246
CV fold 6
	score  fan error       time
	train       0.13  70.041623
	valid       0.39  14.19

In [3]:
!ramp_test_submission --submission haiyen_submission_4

Testing Prediction of annual revenue using FAN
Reading train and test files from ./data ...
Reading cv ...
Training submissions\haiyen_submission_4 ...
CV fold 0
	score  fan error       time
	train       0.15  83.321746
	valid       0.39  17.919608
	test        0.38   9.693214
CV fold 1
	score  fan error       time
	train       0.15  83.941555
	valid       0.39  17.952746
	test        0.38  10.020159
CV fold 2
	score  fan error       time
	train       0.14  81.673289
	valid       0.39  18.008927
	test        0.37   9.976913
CV fold 3
	score  fan error       time
	train       0.14  82.052088
	valid       0.39  18.047013
	test        0.38  10.035290
CV fold 4
	score  fan error       time
	train       0.17  83.762078
	valid       0.41  17.794741
	test        0.39  10.539607
CV fold 5
	score  fan error       time
	train       0.14  84.107375
	valid       0.39  18.000975
	test        0.38  10.041177
CV fold 6
	score  fan error       time
	train       0.14  84.618323
	valid       0.39  17.96

In [4]:
!ramp_test_submission --submission haiyen_submission_5

Testing Prediction of annual revenue using FAN
Reading train and test files from ./data ...
Reading cv ...
Training submissions\haiyen_submission_5 ...
CV fold 0
	score  fan error       time
	train       0.15  66.696815
	valid       0.40  16.981612
	test        0.40   9.595214
CV fold 1
	score  fan error       time
	train       0.15  67.931496
	valid       0.40  16.754518
	test        0.39   9.413165
CV fold 2
	score  fan error       time
	train       0.14  66.213099
	valid       0.40  16.980257
	test        0.39   9.597155
CV fold 3
	score  fan error       time
	train       0.15  66.554931
	valid       0.41  16.979525
	test        0.39   9.354184
CV fold 4
	score  fan error       time
	train       0.15  66.529934
	valid       0.40  17.201379
	test        0.39   9.298321
CV fold 5
	score  fan error       time
	train       0.14  66.252864
	valid       0.40  17.587882
	test        0.39   9.315075
CV fold 6
	score  fan error       time
	train       0.14  67.140856
	valid       0.40  17.32

In [5]:
!ramp_test_submission --submission haiyen_submission_6

Testing Prediction of annual revenue using FAN
Reading train and test files from ./data ...
Reading cv ...
Training submissions\haiyen_submission_6 ...
CV fold 0
	score  fan error       time
	train       0.13  69.970058
	valid       0.38  17.007150
	test        0.39   9.430250
CV fold 1
	score  fan error       time
	train       0.14  69.726557
	valid       0.39  17.035280
	test        0.37   9.231528
CV fold 2
	score  fan error       time
	train       0.13  67.877645
	valid       0.39  16.855330
	test        0.37   9.522130
CV fold 3
	score  fan error       time
	train       0.15  68.644936
	valid       0.40  17.028595
	test        0.37   9.612817
CV fold 4
	score  fan error       time
	train       0.14  69.101283
	valid       0.39  16.933542
	test        0.38   9.268121
CV fold 5
	score  fan error       time
	train       0.13  67.550004
	valid       0.39  16.967211
	test        0.38   9.213238
CV fold 6
	score  fan error       time
	train       0.13  69.305604
	valid       0.39  17.24